In [ ]:
## Notebook for trimming raw SAXS data

import os
import pandas as pd

saxs_file_dir = "./SAXS_Data"

input_files = []
skip_files = []
for root, dirs, files in os.walk(saxs_file_dir, topdown=False):
    for name in files:
        fp = os.path.join(root, name)
        print(fp)

        if fp.endswith(".dat"):
            input_files.append(fp)
        else:
            skip_files.append(fp)


# helper function to append to filename
def dat_filename_append(input_fp, append):
    if input_fp.endswith(".dat"):
        return input_fp[:-4] + f"_{append}" + ".dat"
    else:
        raise ValueError("Incorrect input filepath!")


n_rows_to_remove = 35
q_val_threshold = 0.4

input_files_trimmed = []
out_dir = "./lpla_trimmed/"
os.makedirs(out_dir, exist_ok=True)

for fp in input_files:
    date = fp.split("/")[2].split("_")[
        0
    ]  # get date from filepath (change if files are named differently)
    out_fp = os.path.basename(fp)
    out_fp = os.path.join(out_dir, date + "_" + dat_filename_append(out_fp, "trimmed"))

    # read in data file
    df = pd.read_csv(fp, sep="\t", header=None)
    trim_idx = len(df)

    # find end index
    for i in range(len(df)):
        if df.iloc[i][0] > q_val_threshold:
            trim_idx = i
            break

    # trim data to end
    df_trimmed = df.iloc[n_rows_to_remove:trim_idx]

    #save trimmed data
    df_trimmed.to_csv(out_fp, sep="\t", index=False, header=False)